In [3]:
import pickle

# 1. Carica X_test dal file pickle
with open('Export\Variabili Modello per metriche\Variabili Modello Minimizzato\X_test_MIN.pkl', 'rb') as file:
    X_test = pickle.load(file)

# 2. Carica y_pred dal file pickle
with open('Export\Variabili Modello per metriche\Variabili Modello Minimizzato\y_pred_MIN.pkl', 'rb') as file:
    y_pred = pickle.load(file)

# 3. Carica y_test dal file pickle
with open('Export\Variabili Modello per metriche\Variabili Modello Minimizzato\y_test.pkl', 'rb') as file:
    y_test = pickle.load(file)

# 4. Carica y_test dal file pickle
with open('Export\Dataframe\df_cv_job_PreProc.pkl', 'rb') as file:
    df = pickle.load(file)  

# Verifica i dati caricati
X_test


,age_cv,competences_coverage_required,competences_coverage_optional,education_cv_encoded,education_job_req,education_job_encoded,gender_job_Any,gender_job_Female,gender_job_Male,gender_cv_Female,...,nationality_cv_German,nationality_cv_Irish,nationality_cv_Italian,nationality_cv_LowGerman,nationality_cv_Romanian,nationality_cv_Russian,nationality_cv_Spanish,nationality_cv_Swedish,age_min_job,age_max_job
5000,54,0.250000,0.000000,1,1,1,False,False,True,False,...,False,False,False,False,False,False,True,False,22,37
5001,26,0.500000,0.000000,2,1,1,False,False,True,True,...,False,False,False,False,False,False,True,False,22,37
5002,16,0.250000,0.000000,0,1,1,False,False,True,False,...,False,False,False,False,False,False,False,False,22,37
5003,53,1.000000,0.000000,3,1,1,False,False,True,False,...,False,False,False,False,False,False,False,False,22,37
5004,43,0.250000,0.000000,2,1,1,False,False,True,True,...,False,False,False,False,False,True,False,False,22,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,26,1.000000,0.666667,2,1,1,False,False,True,False,...,False,False,True,False,False,False,False,False,18,34
24996,32,1.000000,0.333333,2,1,1,False,False,True,False,...,False,False,False,False,False,False,True,False,18,34
24997,29,1.000000,0.333333,2,1,1,False,False,True,False,...,False,False,True,False,False,False,False,False,18,34
24998,21,0.333333,0.000000,1,1,1,False,False,True,False,...,False,False,False,False,False,False,False,False,18,34


In [4]:
list(X_test.columns)

['age_cv',
 'competences_coverage_required',
 'competences_coverage_optional',
 'education_cv_encoded',
 'education_job_req',
 'education_job_encoded',
 'gender_job_Any',
 'gender_job_Female',
 'gender_job_Male',
 'gender_cv_Female',
 'gender_cv_Male',
 'gender_cv_Not binary',
 'gender_cv_Not declared',
 'disability_cv_No',
 'disability_cv_Not declared',
 'disability_cv_Yes',
 'disability_job_Any',
 'disability_job_No',
 'PCA_1',
 'PCA_2',
 'nationality_job_Any',
 'nationality_job_Catalan',
 'nationality_job_English',
 'nationality_job_French',
 'nationality_job_Italian',
 'nationality_job_Spanish',
 'nationality_cv_Catalan',
 'nationality_cv_English',
 'nationality_cv_French',
 'nationality_cv_German',
 'nationality_cv_Irish',
 'nationality_cv_Italian',
 'nationality_cv_LowGerman',
 'nationality_cv_Romanian',
 'nationality_cv_Russian',
 'nationality_cv_Spanish',
 'nationality_cv_Swedish',
 'age_min_job',
 'age_max_job']

In [5]:
y_test.value_counts()

isConsistent
0    3000
1    2000
Name: count, dtype: int64

In [6]:
y_pred

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

Per calcolare le metriche rilevanti al nostro scopo, utilizzeremo le librerie Fairlearn.
Per le metriche privacy, creeremo delle funzioni ad-hoc.

Fairlearn utilizza il parametro "sensitive_features" per indicare la feature sensibili sulla quale la metrica è valutata.

Le nostre feature sensibili sono : gender, age, disability, nationality, contract e education



In [7]:
from fairlearn.metrics import demographic_parity_difference
from fairlearn.metrics import demographic_parity_ratio, MetricFrame, selection_rate
import pandas as pd

#### Ci servono le feature non "one-hot encoded", quindi creiamo delle funzioni per combinare le variabili binarie in un'unica feature (one-hot-encoding inverso)

In [8]:
def combine_gender_columns(row):
    if row['gender_cv_Female'] == 1:
        return 'Female'
    elif row['gender_cv_Male'] == 1:
        return 'Male'
    elif row['gender_cv_Not binary'] == 1:
        return 'Not binary'
    elif row['gender_cv_Not declared'] == 1:
        return 'Not declared'
    else:
        return 'Unknown'

X_test['gender'] = X_test.apply(combine_gender_columns, axis=1)

In [9]:
					
def combine_nationality_columns(row):
    if row['nationality_cv_Catalan'] == True:
        return 'Catalan'
    elif row ['nationality_cv_English'] == True:
            return 'English'
    elif row ['nationality_cv_French'] == True:
        return 'French'
    elif row ['nationality_cv_German'] == True :
        return 'German'
    elif row ['nationality_cv_Irish'] == True:
        return 'Irish'
    elif row ['nationality_cv_Italian'] == True:
        return 'Italian'
    elif row ['nationality_cv_LowGerman'] == True:
        return 'LowGerman'
    elif row ['nationality_cv_Romanian'] == True:
        return 'Romanian'
    elif row ['nationality_cv_Russian'] == True:
        return 'Russian'
    elif row ['nationality_cv_Spanish'] == True:
        return 'Spanish'
    elif row ['nationality_cv_Swedish'] == True:
        return 'Swedish'
    
X_test['nationality'] = X_test.apply(combine_nationality_columns, axis=1)
X_test['nationality'].value_counts()

nationality
Italian      934
Spanish      902
French       868
LowGerman    369
Russian      323
English      293
Irish        292
Swedish      292
German       249
Romanian     241
Catalan      237
Name: count, dtype: int64

In [10]:
def combine_disability_columns(row):
    if row['disability_cv_No'] == 1:
        return 'No'
    elif row['disability_cv_Yes'] == 1:
        return 'Yes'
    elif row['disability_cv_Not declared'] == 1:
        return 'Not declared'
    else:
        return 'Unknown'
X_test['disability'] = X_test.apply(combine_disability_columns, axis=1)

In [11]:
# Age (creiamo una versione categorica per facilità)
age_bins = [16, 25, 35, 45, 55, 61]  # Fasce di età specificate
age_labels = ['16-25', '26-35', '36-45', '46-55', '56-60']
X_test['age_group'] = pd.cut(X_test['age_cv'], bins=age_bins, labels=age_labels, right=False)

X_test['age_group'].value_counts()

age_group
26-35    1996
16-25     875
36-45     867
46-55     755
56-60     507
Name: count, dtype: int64

In [12]:
X_test['age_group'] = X_test['age_group'].astype(str)

### RIesportiamo X_test per usarlo dopo per recuperare i nomi delle colonne modificati

In [13]:
with open('X_test_MIN_perPlot.pkl', 'wb') as file:
    pickle.dump(X_test, file)

In [14]:
sensitive_variables=['gender', 'age_group', 'disability', 'nationality', 'education_cv_encoded']

In [15]:
# Definire i gruppi sensibili come una lista di variabili
sensitive_features = X_test[sensitive_variables]

Inizializziamo dizionario che accoglierà tutte le metriche , che verrà poi convertito in un Dataframe che verrà poi utilizzato per la visualizzazione delle metriche

In [16]:
results = {'Metric':[]}

Creiamo una key per ogni variabile sensibile

In [17]:
for key in sensitive_variables:
    results[key] = []

Creiamo una key per ogni valore univoco delle variabili sensibili. Ci servirà poichè alcune metriche vengono calcolate sui valori piuttosto che su tutta la feature

In [18]:
# Lista per salvare i nomi delle nuove chiavi
valori_univoci = []

In [19]:
for col in sensitive_variables:
    for unique_value in X_test[col].unique():
        key = f"{col}_{unique_value}"  # Creare la chiave con il prefisso
        results[key] = []  # Inizializzare con una lista vuota
        valori_univoci.append(key)  # Salvare il nome della chiave nella lista

In [20]:
print(results)

{'Metric': [], 'gender': [], 'age_group': [], 'disability': [], 'nationality': [], 'education_cv_encoded': [], 'gender_Not declared': [], 'gender_Female': [], 'gender_Male': [], 'gender_Not binary': [], 'age_group_46-55': [], 'age_group_26-35': [], 'age_group_16-25': [], 'age_group_36-45': [], 'age_group_56-60': [], 'disability_No': [], 'disability_Yes': [], 'disability_Not declared': [], 'nationality_Spanish': [], 'nationality_English': [], 'nationality_Russian': [], 'nationality_Italian': [], 'nationality_Irish': [], 'nationality_Catalan': [], 'nationality_French': [], 'nationality_German': [], 'nationality_LowGerman': [], 'nationality_Swedish': [], 'nationality_Romanian': [], 'education_cv_encoded_1': [], 'education_cv_encoded_2': [], 'education_cv_encoded_0': [], 'education_cv_encoded_3': [], 'education_cv_encoded_4': []}


Definiamo funzione utile per la creazione della lista 

In [21]:
def append_none_to_specified_keys(dictionary, keys):
    """
    Aggiunge None alla lista associata solo alle chiavi specificate.
    Ignora le chiavi il cui valore non è una lista o che non esistono nel dizionario.
    
    :param dictionary: dict, il dizionario da modificare
    :param keys: list, l'elenco delle chiavi a cui aggiungere None
    """
    for key in keys:
        if key in dictionary and isinstance(dictionary[key], list):  # Verifica esistenza e tipo lista
            dictionary[key].append(None)

## Demographic parity difference

#### The demographic parity difference is defined as the difference between the largest and the smallest group-level selection rate, across all values of the sensitive feature(s). The demographic parity difference of 0 means that all groups have the same selection rate.

In [22]:
results['Metric'].append('Demographic parity difference')

In [23]:


for feature in sensitive_features:
    dp_diff = demographic_parity_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    results[feature].append(1-dp_diff)
    print(f"\nDemographic parity difference per '{feature}':")
    print(dp_diff)


Demographic parity difference per 'gender':
0.4488722427642484

Demographic parity difference per 'age_group':
0.536

Demographic parity difference per 'disability':
0.2512698293974855

Demographic parity difference per 'nationality':
0.3863819500402901

Demographic parity difference per 'education_cv_encoded':
0.313604756356887


In [24]:
## Dato che questa metrica è solo per feature, aggiungiamo None alle key inerenti ai valori
append_none_to_specified_keys(results,valori_univoci)

In [25]:
print(results)

{'Metric': ['Demographic parity difference'], 'gender': [0.5511277572357516], 'age_group': [0.46399999999999997], 'disability': [0.7487301706025145], 'nationality': [0.6136180499597099], 'education_cv_encoded': [0.686395243643113], 'gender_Not declared': [None], 'gender_Female': [None], 'gender_Male': [None], 'gender_Not binary': [None], 'age_group_46-55': [None], 'age_group_26-35': [None], 'age_group_16-25': [None], 'age_group_36-45': [None], 'age_group_56-60': [None], 'disability_No': [None], 'disability_Yes': [None], 'disability_Not declared': [None], 'nationality_Spanish': [None], 'nationality_English': [None], 'nationality_Russian': [None], 'nationality_Italian': [None], 'nationality_Irish': [None], 'nationality_Catalan': [None], 'nationality_French': [None], 'nationality_German': [None], 'nationality_LowGerman': [None], 'nationality_Swedish': [None], 'nationality_Romanian': [None], 'education_cv_encoded_1': [None], 'education_cv_encoded_2': [None], 'education_cv_encoded_0': [None

In [26]:
# Creiamo un MetricFrame per calcolare i selection rate per ogni gruppo
selection_rate_gender = MetricFrame(metrics=selection_rate, y_true=y_test, y_pred=y_pred, sensitive_features=X_test['gender'])
selection_rate_age = MetricFrame(metrics=selection_rate, y_true=y_test, y_pred=y_pred, sensitive_features=X_test['age_cv'])
selection_rate_disability = MetricFrame(metrics=selection_rate, y_true=y_test, y_pred=y_pred, sensitive_features=X_test['disability'])
selection_rate_nationality = MetricFrame(metrics=selection_rate, y_true=y_test, y_pred=y_pred, sensitive_features=X_test['nationality'])
selection_rate_education = MetricFrame(metrics=selection_rate, y_true=y_test, y_pred=y_pred, sensitive_features=X_test['education_cv_encoded'])

In [27]:
# Stampa dei selection rate per ogni gruppo
print("\nSelection Rate per ogni gruppo:")
print("Gender:")
print(selection_rate_gender.by_group)
print("\nAge:")
print(selection_rate_age.by_group)
print("\nDisability:")
print(selection_rate_disability.by_group)
print("\nNationality:")
print(selection_rate_nationality.by_group)
print("\nEducation:")
print(selection_rate_education.by_group)


Selection Rate per ogni gruppo:
Gender:
gender
Female          0.237892
Male            0.489545
Not binary      0.040673
Not declared    0.042460
Name: selection_rate, dtype: float64

Age:
age_cv
16    0.000000
17    0.000000
18    0.647727
19    0.576923
20    0.500000
21    0.722689
22    0.556452
23    0.541237
24    0.384615
25    0.525822
26    0.565041
27    0.447581
28    0.441860
29    0.470085
30    0.518868
31    0.321637
32    0.388158
33    0.270677
34    0.379845
35    0.344538
36    0.186047
37    0.236559
38    0.173469
39    0.011905
40    0.000000
41    0.000000
42    0.000000
43    0.000000
44    0.000000
45    0.000000
46    0.000000
47    0.000000
48    0.000000
49    0.000000
50    0.000000
51    0.000000
52    0.000000
53    0.000000
54    0.000000
55    0.000000
56    0.000000
57    0.000000
58    0.000000
59    0.000000
60    0.000000
Name: selection_rate, dtype: float64

Disability:
disability
No              0.361026
Not declared    0.109756
Yes             

## Equalized Odds Difference
#### The greater of two metrics: true_positive_rate_difference and false_positive_rate_difference. The former is the difference between the largest and smallest of (formula)  across all values  "a" of the sensitive feature(s). The latter is defined similarly, but for (formula) The equalized odds difference of 0 means that all groups have the same true positive, true negative, false positive, and false negative rates.

In [28]:
from fairlearn.metrics import equalized_odds_difference

In [29]:
results['Metric'].append('Equalized Odds Difference')

In [30]:

for feature in sensitive_features:
    eod_diff = equalized_odds_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    results[feature].append(1-eod_diff)
    print(f"\nEqualized Odds Difference per '{feature}':")
    print(1-eod_diff)


Equalized Odds Difference per 'gender':
0.5353337682104805

Equalized Odds Difference per 'age_group':
0.25757575757575757

Equalized Odds Difference per 'disability':
0.94512433173188

Equalized Odds Difference per 'nationality':
0.21372282608695647

Equalized Odds Difference per 'education_cv_encoded':
0.44435887562572196


In [31]:
append_none_to_specified_keys(results,valori_univoci)

In [32]:
print(results)

{'Metric': ['Demographic parity difference', 'Equalized Odds Difference'], 'gender': [0.5511277572357516, 0.5353337682104805], 'age_group': [0.46399999999999997, 0.25757575757575757], 'disability': [0.7487301706025145, 0.94512433173188], 'nationality': [0.6136180499597099, 0.21372282608695647], 'education_cv_encoded': [0.686395243643113, 0.44435887562572196], 'gender_Not declared': [None, None], 'gender_Female': [None, None], 'gender_Male': [None, None], 'gender_Not binary': [None, None], 'age_group_46-55': [None, None], 'age_group_26-35': [None, None], 'age_group_16-25': [None, None], 'age_group_36-45': [None, None], 'age_group_56-60': [None, None], 'disability_No': [None, None], 'disability_Yes': [None, None], 'disability_Not declared': [None, None], 'nationality_Spanish': [None, None], 'nationality_English': [None, None], 'nationality_Russian': [None, None], 'nationality_Italian': [None, None], 'nationality_Irish': [None, None], 'nationality_Catalan': [None, None], 'nationality_Fren

TODO: stampare i diversi rate

## Equal Opportunity difference

#### Misura il True Positive Rate fra i diversi gruppi (se è uguale è OK)
#### Non c'è una metrica ad-hoc per questa misura, ma utilizziamo il true_positive_rate per ciascun gruppo e poi calcoliamo la differenza

In [33]:
results['Metric'].append('Equal Opportunity difference')

In [34]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import recall_score, precision_score
import pandas as pd

In [35]:
for feature in sensitive_features:
    metric_frame = MetricFrame(
        metrics=recall_score,
        y_true=y_test,
        y_pred=y_pred,
        sensitive_features=X_test[feature]
    )
    tpr_difference= metric_frame.difference()
    results[feature].append(1-tpr_difference)
    print(f"\nTPR Difference per '{feature}':")
    print(1-tpr_difference)


TPR Difference per 'gender':
0.5353337682104805

TPR Difference per 'age_group':
0.25757575757575757


c:\Users\marco\Documents\UniPI\Tesi\PJ\S0_MidBias\Synthetic_cv-job_generator\.conda\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marco\Documents\UniPI\Tesi\PJ\S0_MidBias\Synthetic_cv-job_generator\.conda\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



TPR Difference per 'disability':
0.981447963800905

TPR Difference per 'nationality':
0.21372282608695647

TPR Difference per 'education_cv_encoded':
0.44435887562572196


In [36]:
append_none_to_specified_keys(results,valori_univoci)

In [37]:
print(results)

{'Metric': ['Demographic parity difference', 'Equalized Odds Difference', 'Equal Opportunity difference'], 'gender': [0.5511277572357516, 0.5353337682104805, 0.5353337682104805], 'age_group': [0.46399999999999997, 0.25757575757575757, 0.25757575757575757], 'disability': [0.7487301706025145, 0.94512433173188, 0.981447963800905], 'nationality': [0.6136180499597099, 0.21372282608695647, 0.21372282608695647], 'education_cv_encoded': [0.686395243643113, 0.44435887562572196, 0.44435887562572196], 'gender_Not declared': [None, None, None], 'gender_Female': [None, None, None], 'gender_Male': [None, None, None], 'gender_Not binary': [None, None, None], 'age_group_46-55': [None, None, None], 'age_group_26-35': [None, None, None], 'age_group_16-25': [None, None, None], 'age_group_36-45': [None, None, None], 'age_group_56-60': [None, None, None], 'disability_No': [None, None, None], 'disability_Yes': [None, None, None], 'disability_Not declared': [None, None, None], 'nationality_Spanish': [None, N

#### Il TPR Difference sarà calcolato come la differenza tra il TPR del gruppo con il tasso di veri positivi più alto e il TPR del gruppo con il tasso di veri positivi più basso.

## Disparate Impact 
#### Rapporto tra le probabilità di assegnazione a una classe positiva tra gruppi non privilegiati e privilegiati. Corrisponde allo stesso calcolo della metrica Demographic parity Ratio

#### avendo già demographic parity Difference forse meglio elimanre ? 

In [38]:
#results['Metric'].append('Disparate Impact')

In [39]:
#for feature in sensitive_features:
 #   dp_ratio = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
  #  results[feature].append(1-dp_ratio)
   # print(f"\nDisparate Impact per '{feature}':")
    #print(dp_ratio)

In [40]:
#append_none_to_specified_keys(results,valori_univoci)

In [41]:
#print(results)

## Predictive Parity

In [42]:
results['Metric'].append('Predictive Parity Difference')
results['Metric'].append('Predictive Parity')


A fairness metric that checks whether, for a given classifier, the precision rates are equivalent for subgroups under consideration.

In [43]:
append_none_to_specified_keys(results,valori_univoci)
for feature in sensitive_features:
    metric_frame = MetricFrame(
        metrics=precision_score,
        y_true=y_test,
        y_pred=y_pred,
        sensitive_features=X_test[feature]
    )
    precision_ratio= metric_frame.ratio()
    difference=metric_frame.difference()

    print("Precision per gruppo (Predictive Parity):")
    print(f"\n '{feature}':")
    print(metric_frame.by_group)
    print(f"\nRapporto di Predictive parity per '{feature}':")
    print(precision_ratio)
    print(f"\nDifferenza di Predictive parity per '{feature}':")
    print(difference)
    results[feature].append(1-difference)
    for index, value in metric_frame.by_group.items():
        key = f"{feature}_{index}"
        results[key].append(value)
         
    


Precision per gruppo (Predictive Parity):

 'gender':
gender
Female          0.898204
Male            0.915506
Not binary      0.793103
Not declared    0.827586
Name: precision_score, dtype: float64

Rapporto di Predictive parity per 'gender':
0.8663006225082185

Differenza di Predictive parity per 'gender':
0.12240258700733198
Precision per gruppo (Predictive Parity):

 'age_group':
age_group
16-25    0.940299
26-35    0.910615
36-45    0.733333
46-55    0.000000
56-60    0.000000
Name: precision_score, dtype: float64

Rapporto di Predictive parity per 'age_group':
0.0

Differenza di Predictive parity per 'age_group':
0.9402985074626866


c:\Users\marco\Documents\UniPI\Tesi\PJ\S0_MidBias\Synthetic_cv-job_generator\.conda\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marco\Documents\UniPI\Tesi\PJ\S0_MidBias\Synthetic_cv-job_generator\.conda\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision per gruppo (Predictive Parity):

 'disability':
disability
No              0.912741
Not declared    0.944444
Yes             0.847826
Name: precision_score, dtype: float64

Rapporto di Predictive parity per 'disability':
0.8976982097186701

Differenza di Predictive parity per 'disability':
0.09661835748792269
Precision per gruppo (Predictive Parity):

 'nationality':
nationality
Catalan      0.800000
English      0.881720
French       0.921212
German       0.636364
Irish        0.953846
Italian      0.942149
LowGerman    0.692308
Romanian     0.720000
Russian      0.631579
Spanish      0.921875
Swedish      0.935484
Name: precision_score, dtype: float64

Rapporto di Predictive parity per 'nationality':
0.6621392190152801

Differenza di Predictive parity per 'nationality':
0.3222672064777329
Precision per gruppo (Predictive Parity):

 'education_cv_encoded':
education_cv_encoded
0    0.521739
1    0.922360
2    0.908451
3    0.926290
4    0.285714
Name: precision_score, dtype:

In [44]:
append_none_to_specified_keys(results,sensitive_variables)

In [45]:
print(results)

{'Metric': ['Demographic parity difference', 'Equalized Odds Difference', 'Equal Opportunity difference', 'Predictive Parity Difference', 'Predictive Parity'], 'gender': [0.5511277572357516, 0.5353337682104805, 0.5353337682104805, 0.877597412992668, None], 'age_group': [0.46399999999999997, 0.25757575757575757, 0.25757575757575757, 0.05970149253731338, None], 'disability': [0.7487301706025145, 0.94512433173188, 0.981447963800905, 0.9033816425120773, None], 'nationality': [0.6136180499597099, 0.21372282608695647, 0.21372282608695647, 0.6777327935222671, None], 'education_cv_encoded': [0.686395243643113, 0.44435887562572196, 0.44435887562572196, 0.35942435942435946, None], 'gender_Not declared': [None, None, None, None, 0.8275862068965517], 'gender_Female': [None, None, None, None, 0.8982035928143712], 'gender_Male': [None, None, None, None, 0.9155060352831941], 'gender_Not binary': [None, None, None, None, 0.7931034482758621], 'age_group_46-55': [None, None, None, None, 0.0], 'age_group

## Treatment Equality
The treatment equality (TE) is the difference in the ratio of false negatives to false positives

In [46]:
#results['Metric'].append('Treatment Equality Difference')
#results['Metric'].append('Treatment Equality')


In [47]:
from fairlearn.metrics import false_positive_rate, false_negative_rate

In [48]:
#append_none_to_specified_keys(results,valori_univoci)
# Dizionario per salvare i risultati di Treatment Equality
#treatment_equalities = {}

# Calcola FPR, FNR e Treatment Equality per ciascuna variabile sensibile
"""for feature in sensitive_features:
    fpr = MetricFrame(metrics=false_positive_rate, y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    fnr = MetricFrame(metrics=false_negative_rate, y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])
    
    treatment_equality = fpr.by_group / fnr.by_group
    treatment_equalities[feature] = treatment_equality

# Visualizza i risultati
for feature, treatment_equality in treatment_equalities.items():
    print(f"Treatment Equality per {feature.capitalize()}:")
    print(treatment_equality)
    print("\n")
    # Calcola la differenza tra il massimo e il minimo
    max_ratio = treatment_equality.max()
    min_ratio = treatment_equality.min()
    diff_max_min = max_ratio - min_ratio
    results[feature].append(diff_max_min)
    for index, value in treatment_equality.items():
        key = f"{feature}_{index}"
        if (value > 1):
            results[key].append(1-value)
        else:
            results[key].append(value)"""
         

'for feature in sensitive_features:\n    fpr = MetricFrame(metrics=false_positive_rate, y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])\n    fnr = MetricFrame(metrics=false_negative_rate, y_true=y_test, y_pred=y_pred, sensitive_features=X_test[feature])\n    \n    treatment_equality = fpr.by_group / fnr.by_group\n    treatment_equalities[feature] = treatment_equality\n\n# Visualizza i risultati\nfor feature, treatment_equality in treatment_equalities.items():\n    print(f"Treatment Equality per {feature.capitalize()}:")\n    print(treatment_equality)\n    print("\n")\n    # Calcola la differenza tra il massimo e il minimo\n    max_ratio = treatment_equality.max()\n    min_ratio = treatment_equality.min()\n    diff_max_min = max_ratio - min_ratio\n    results[feature].append(diff_max_min)\n    for index, value in treatment_equality.items():\n        key = f"{feature}_{index}"\n        if (value > 1):\n            results[key].append(1-value)\n        else:\n           

In [49]:
#append_none_to_specified_keys(results,sensitive_variables)

In [50]:
#print(results)

## Overall accuracy equality

## Accuratezza complessiva fra gruppi

In [51]:
results['Metric'].append('Overall accuracy equality difference')
results['Metric'].append('Overall accuracy equality')

In [52]:
from sklearn.metrics import accuracy_score

append_none_to_specified_keys(results,valori_univoci)
for feature in sensitive_features:
    metric_frame = MetricFrame(
        metrics=accuracy_score,
        y_true=y_test,
        y_pred=y_pred,
        sensitive_features=X_test[feature]
    )
    difference=metric_frame.difference()
    print(f"\nOverall Accuracy Equality per gruppo '{feature}':")
    print(metric_frame.by_group)
    print(f"\nDifferenza di Predictive parity per '{feature}':")
    print(difference)
    results[feature].append(1-difference)
    for index, value in metric_frame.by_group.items():
        key = f"{feature}_{index}"
        results[key].append(value)


Overall Accuracy Equality per gruppo 'gender':
gender
Female          0.811966
Male            0.809545
Not binary      0.922861
Not declared    0.904832
Name: accuracy_score, dtype: float64

Differenza di Predictive parity per 'gender':
0.11331569552467169

Overall Accuracy Equality per gruppo 'age_group':
age_group
16-25    0.793143
26-35    0.728958
36-45    0.906574
46-55    1.000000
56-60    1.000000
Name: accuracy_score, dtype: float64

Differenza di Predictive parity per 'age_group':
0.27104208416833664

Overall Accuracy Equality per gruppo 'disability':
disability
No              0.805130
Not declared    0.939024
Yes             0.922581
Name: accuracy_score, dtype: float64

Differenza di Predictive parity per 'disability':
0.1338947554515968

Overall Accuracy Equality per gruppo 'nationality':
nationality
Catalan      0.767932
English      0.750853
French       0.907834
German       0.710843
Irish        0.886986
Italian      0.932548
LowGerman    0.718157
Romanian     0.7136

In [53]:
append_none_to_specified_keys(results,sensitive_variables)

In [54]:
print(results)

{'Metric': ['Demographic parity difference', 'Equalized Odds Difference', 'Equal Opportunity difference', 'Predictive Parity Difference', 'Predictive Parity', 'Overall accuracy equality difference', 'Overall accuracy equality'], 'gender': [0.5511277572357516, 0.5353337682104805, 0.5353337682104805, 0.877597412992668, None, 0.8866843044753283, None], 'age_group': [0.46399999999999997, 0.25757575757575757, 0.25757575757575757, 0.05970149253731338, None, 0.7289579158316634, None], 'disability': [0.7487301706025145, 0.94512433173188, 0.981447963800905, 0.9033816425120773, None, 0.8661052445484032, None], 'nationality': [0.6136180499597099, 0.21372282608695647, 0.21372282608695647, 0.6777327935222671, None, 0.7268945445866839, None], 'education_cv_encoded': [0.686395243643113, 0.44435887562572196, 0.44435887562572196, 0.35942435942435946, None, 0.8651871126418549, None], 'gender_Not declared': [None, None, None, None, 0.8275862068965517, None, 0.9048316251830161], 'gender_Female': [None, No

## Conditional Statistical parity

#### calcola la parità di assegnazione dei gruppi ad una classe positiva tenendo un set di caratteristiche uguale ( qua viene applicato prendendo la copertura di competenze richieste) : praticamente vogliamo vedere se a parità di competenze ci sono disparità di assegnazione fra i gruppi

In [55]:
y_test

5000     0
5001     0
5002     0
5003     0
5004     0
        ..
24995    1
24996    1
24997    1
24998    1
24999    1
Name: isConsistent, Length: 5000, dtype: int64

In [56]:
y_pred.shape

(5000,)

# TODO : SALVARE QUESTA METRICA IN ALTRO MODO PER POI VISUALIZZARLA

In [57]:
# Allineare gli indici di y_test e y_pred con X_test
y_test = y_test.reset_index(drop=True)
y_pred = pd.Series(y_pred)
y_pred = y_pred.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

conditional_variable = 'competences_coverage_required'

# Inizializziamo un dizionario per contenere i risultati
csp_results = {}


# Iteriamo sui gruppi definiti dalla variabile `competences_coverage_required`
for condition in X_test[conditional_variable].unique():
    subset = X_test[X_test[conditional_variable] == condition]  # Filtriamo per ogni gruppo di esperienza
    y_true_subset = y_test[subset.index]
    y_pred_subset = y_pred[subset.index]
    results['Metric'].append(f"CSP_{conditional_variable}' = {condition:.2f}_difference")
    append_none_to_specified_keys(results,valori_univoci)
    #results['Metric'].append(f"CSP_{conditional_variable}' = {condition}")
    
    # Calcoliamo la parità statistica per ciascuna variabile sensibile nel gruppo condizionato
    for feature in sensitive_features:
        metric_frame = MetricFrame(
            metrics=selection_rate,  # selection_rate misura la probabilità di una previsione positiva
            y_true=y_true_subset,
            y_pred=y_pred_subset,
            sensitive_features=subset[feature]
        )
        
        # Salviamo i risultati
        if feature not in csp_results:
            csp_results[feature] = {}
        csp_results[feature][condition] = metric_frame.by_group

# Visualizziamo i risultati
for feature, result in csp_results.items():
    print(f"\nConditional Statistical Parity per la feature sensibile '{feature}':")
    for condition, values in result.items():
        print(f"\nCondizione '{conditional_variable}' = {condition:.2f}:")
        print(values)
        max_ratio = values.max()
        min_ratio = values.min()
        diff_max_min = max_ratio - min_ratio
        results[feature].append(1-diff_max_min)
        print(f"\nDifferenza di CSP = '{condition}' per '{feature}':")
        print(diff_max_min)
        #Dato che alcuni valori non sono rappresentati, non salviamo questa metrica per valore ma solo per differenza globale
        #for index, valori in values.items():
           # key = f"{feature}_{index}"
            #results[key].append(valori)
        
        


Conditional Statistical Parity per la feature sensibile 'gender':

Condizione 'competences_coverage_required' = 0.25:
gender
Female          0.150183
Male            0.168539
Not binary      0.017143
Not declared    0.027397
Name: selection_rate, dtype: float64

Differenza di CSP = '0.25' per 'gender':
0.15139646869983947

Condizione 'competences_coverage_required' = 0.50:
gender
Female          0.267003
Male            0.320755
Not binary      0.060606
Not declared    0.025773
Name: selection_rate, dtype: float64

Differenza di CSP = '0.5' per 'gender':
0.2949815211048434

Condizione 'competences_coverage_required' = 1.00:
gender
Female          0.299539
Male            0.602122
Not binary      0.061905
Not declared    0.100000
Name: selection_rate, dtype: float64

Differenza di CSP = '1.0' per 'gender':
0.5402172540103575

Condizione 'competences_coverage_required' = 0.75:
gender
Female          0.190000
Male            0.070381
Not binary      0.018405
Not declared    0.012270
Name

In [58]:
print(results)

{'Metric': ['Demographic parity difference', 'Equalized Odds Difference', 'Equal Opportunity difference', 'Predictive Parity Difference', 'Predictive Parity', 'Overall accuracy equality difference', 'Overall accuracy equality', "CSP_competences_coverage_required' = 0.25_difference", "CSP_competences_coverage_required' = 0.50_difference", "CSP_competences_coverage_required' = 1.00_difference", "CSP_competences_coverage_required' = 0.75_difference", "CSP_competences_coverage_required' = 0.33_difference", "CSP_competences_coverage_required' = 0.67_difference"], 'gender': [0.5511277572357516, 0.5353337682104805, 0.5353337682104805, 0.877597412992668, None, 0.8866843044753283, None, 0.8486035313001605, 0.7050184788951566, 0.4597827459896425, 0.8222699386503067, 1.0, 1.0], 'age_group': [0.46399999999999997, 0.25757575757575757, 0.25757575757575757, 0.05970149253731338, None, 0.7289579158316634, None, 0.752895752895753, 0.4832535885167464, 0.3265895953757225, 0.8, 0.5169491525423728, 0.767441

## Conditional Use Accuracy Equality

#### si controlla che l'accuratezza condizionata ( nel nostro casa l'accuratezza su isConsistent=1) sia uguale fra i gruppi sensibili

In [59]:
results['Metric'].append('Conditional Use Accuracy Equality (isConsistent=1)_difference')
results['Metric'].append('Conditional Use Accuracy Equality (isConsistent=1)')

In [60]:
# Filtriamo i dati per considerare solo i casi con predizione positiva
positive_predictions = y_pred == 1
y_test_positive = y_test[positive_predictions]
y_pred_positive = y_pred[positive_predictions]
X_test_positive = X_test[positive_predictions]

# Inizializziamo un dizionario per memorizzare i risultati di accuracy per ogni feature sensibile
cuae_results = {}
append_none_to_specified_keys(results,valori_univoci)
# Calcoliamo l'accuratezza per ciascun gruppo sensibile
for feature in sensitive_features:
    metric_frame = MetricFrame(
        metrics=accuracy_score,
        y_true=y_test_positive,
        y_pred=y_pred_positive,
        sensitive_features=X_test_positive[feature]
    )
    
    cuae_results[feature] = metric_frame.by_group

# Visualizziamo i risultati
for feature, result in cuae_results.items():
    print(f"\nConditional Use Accuracy Equality per la feature sensibile '{feature}' (solo predizioni positive):")
    print(result)
    max_ratio = result.max()
    min_ratio = result.min()
    diff_max_min = max_ratio - min_ratio
    results[feature].append(1-diff_max_min)
    print(f"\nConditional Use Accuracy Equality Difference per la feature sensibile '{feature}' (solo predizioni positive):")
    print(diff_max_min)
    for index, value in result.items():
        key = f"{feature}_{index}"
        results[key].append(value)
    


Conditional Use Accuracy Equality per la feature sensibile 'gender' (solo predizioni positive):
gender
Female          0.898204
Male            0.915506
Not binary      0.793103
Not declared    0.827586
Name: accuracy_score, dtype: float64

Conditional Use Accuracy Equality Difference per la feature sensibile 'gender' (solo predizioni positive):
0.12240258700733198

Conditional Use Accuracy Equality per la feature sensibile 'age_group' (solo predizioni positive):
age_group
16-25    0.940299
26-35    0.910615
36-45    0.733333
Name: accuracy_score, dtype: float64

Conditional Use Accuracy Equality Difference per la feature sensibile 'age_group' (solo predizioni positive):
0.20696517412935334

Conditional Use Accuracy Equality per la feature sensibile 'disability' (solo predizioni positive):
disability
No              0.912741
Not declared    0.944444
Yes             0.847826
Name: accuracy_score, dtype: float64

Conditional Use Accuracy Equality Difference per la feature sensibile 'dis

In [61]:
append_none_to_specified_keys(results,sensitive_variables)


In [62]:
append_none_to_specified_keys(results,["age_group_56-60","age_group_46-55"]) #da valutare se tenerli oppure non calcolare la metrica per valori

In [63]:
print(results)

{'Metric': ['Demographic parity difference', 'Equalized Odds Difference', 'Equal Opportunity difference', 'Predictive Parity Difference', 'Predictive Parity', 'Overall accuracy equality difference', 'Overall accuracy equality', "CSP_competences_coverage_required' = 0.25_difference", "CSP_competences_coverage_required' = 0.50_difference", "CSP_competences_coverage_required' = 1.00_difference", "CSP_competences_coverage_required' = 0.75_difference", "CSP_competences_coverage_required' = 0.33_difference", "CSP_competences_coverage_required' = 0.67_difference", 'Conditional Use Accuracy Equality (isConsistent=1)_difference', 'Conditional Use Accuracy Equality (isConsistent=1)'], 'gender': [0.5511277572357516, 0.5353337682104805, 0.5353337682104805, 0.877597412992668, None, 0.8866843044753283, None, 0.8486035313001605, 0.7050184788951566, 0.4597827459896425, 0.8222699386503067, 1.0, 1.0, 0.877597412992668, None], 'age_group': [0.46399999999999997, 0.25757575757575757, 0.25757575757575757, 0

## Esportiamo dizionario con le metriche salvate, in modo da riutilizzarlo per il plotting

In [64]:

with open("dicMetric_Fairness_Completed.pkl", "wb") as file:
    pickle.dump(results, file)

In [65]:
# Converti il dizionario in un DataFrame
df = pd.DataFrame(list(results.items()))

# Salva in un file Excel
df.to_excel("dizionario_fairness.xlsx", index=False)

# Privacy Metrics Paper (Technical Privacy Metrics: a Systematic Survey)

In [66]:
results_privacy = {'Metric':[]}
results_privacy['Valore']=[]

## Anonymity set size


Andiamo a calcolare il livello di anonimato di un istanza in un sistema. Si calcolano i value counts delle combinazioni delle feature sensibili per capire, nel caso un avversario avesse accesso ad esse, quanto un individuo è indistinguibile. Calcoliamo poi la media, per dare un'idea generale del livello di anonimato complessivo nel dataset.

TODO: Farlo per sottoinsiemi di combinazioni per simulare l'accesso solo ad alcune di esse

In [67]:
# Imposta le opzioni di visualizzazione per mostrare tutte le righe e colonne
pd.set_option('display.max_rows', None)      # Mostra tutte le righe
pd.set_option('display.max_columns', None)   # Mostra tutte le colonne
pd.set_option('display.width', 1000)         # Larghezza per evitare troncamento

In [68]:
def calculate_anonymity_set_size_for_sensitive_features(sensitive_features):
    
    # Conteggio delle combinazioni uniche delle feature sensibili
    combination_counts = sensitive_features.value_counts()
    
    # Calcolo della dimensione media dell'anonymity set
    average_anonymity_set_size = combination_counts.mean()
    
    return combination_counts, average_anonymity_set_size

# Esempio di utilizzo
combination_counts, average_anonymity_set_size = calculate_anonymity_set_size_for_sensitive_features(sensitive_features)
# Convertiamo combination_counts in un DataFrame per una visualizzazione completa
combination_counts_df = combination_counts.reset_index()
combination_counts_df.columns = list(sensitive_features.columns) + ['count']



# Stampa i risultati in formato tabellare
print("Anonymity set sizes per combinazione di feature sensibili:")
print(combination_counts_df)


Anonymity set sizes per combinazione di feature sensibili:
            gender age_group    disability nationality  education_cv_encoded  count
0             Male     26-35            No     Spanish                     2     82
1             Male     26-35            No     Russian                     2     72
2             Male     26-35            No   LowGerman                     2     70
3             Male     26-35            No     Italian                     2     70
4             Male     26-35            No      French                     2     67
5             Male     16-25            No     Italian                     2     57
6             Male     26-35            No     Italian                     3     54
7             Male     16-25            No      French                     2     50
8             Male     16-25            No     Spanish                     1     48
9             Male     16-25            No     Italian                     1     45
10            Mal

In [69]:
print("\nAverage anonymity set size:", average_anonymity_set_size)


Average anonymity set size: 3.4153005464480874


In [70]:
results_privacy['Metric'].append("Avg Anonimity Set Size")
results_privacy['Valore'].append(average_anonymity_set_size)

In [71]:
# Ripristina le opzioni di visualizzazione se necessario
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')

## K-Anonimity

In [72]:
# Definiamo i quasi-identificatori
quasi_identifiers = ['age_group', 'gender', 'nationality']

# Calcolo del numero di record per ciascun gruppo di quasi-identificatori
k_anonymity_counts = sensitive_features[quasi_identifiers].groupby(quasi_identifiers).size()

# Visualizziamo i gruppi e il loro conteggio
print(k_anonymity_counts)

# Verifichiamo se il dataset soddisfa la K-Anonymity per un valore specifico di k (es. k=2)
k = 1
k_anonymous = k_anonymity_counts >= k
print(f"Soddisfa K-Anonymity per k={k}: {k_anonymous.all()}")

age_group  gender        nationality
16-25      Female        Catalan         7
                         English         9
                         French         26
                         German         10
                         Irish          10
                                        ..
56-60      Not declared  LowGerman       9
                         Romanian        5
                         Russian         7
                         Spanish        20
                         Swedish         6
Length: 220, dtype: int64
Soddisfa K-Anonymity per k=1: True


In [73]:
# Calcola la frequenza delle combinazioni dei quasi-identificatori
quasi_identifiers = ['age_group', 'gender', 'nationality','disability']
k_anonymity = X_test.groupby(quasi_identifiers).size().min()

print(f"La k-anonymity del dataset è: {k_anonymity}")

La k-anonymity del dataset è: 1


In [74]:
results_privacy['Metric'].append("K-Anonimity")
results_privacy['Valore'].append(k_anonymity)

## L-Diversity

In [75]:
# Calcoliamo la diversità dell'attributo sensibile (disability) per ciascun gruppo di quasi-identificatori
l_diversity = X_test.groupby(quasi_identifiers)['disability'].nunique()

# Troviamo il valore minimo di diversità tra tutti i gruppi
l_max_satisfied = l_diversity.min()

# Visualizziamo il valore di l massimo soddisfatto e la diversità per ciascun gruppo
print("L-Diversity per ciascun gruppo:")
print(l_diversity)
print(f"\nIl massimo valore di l per cui il dataset soddisfa L-Diversity è: {l_max_satisfied}")

L-Diversity per ciascun gruppo:
age_group  gender        nationality  disability  
16-25      Female        Catalan      No              1
                         English      No              1
                                      Yes             1
                         French       No              1
                                      Not declared    1
                                                     ..
56-60      Not declared  Russian      Yes             1
                         Spanish      No              1
                                      Yes             1
                         Swedish      No              1
                                      Yes             1
Name: disability, Length: 554, dtype: int64

Il massimo valore di l per cui il dataset soddisfa L-Diversity è: 1


In [76]:
results_privacy['Metric'].append("L-Diversity")
results_privacy['Valore'].append(l_max_satisfied)

## T-Closeness

In [77]:
import pandas as pd
from scipy.stats import wasserstein_distance

In [78]:
# Calcoliamo la distribuzione globale dell'attributo sensibile
global_distribution = X_test['disability'].value_counts(normalize=True).sort_index()

# Funzione per calcolare la t-closeness (Earth Mover's Distance)
def t_closeness(group, global_distribution):
    # Calcola la distribuzione del gruppo
    group_distribution = group['disability'].value_counts(normalize=True).sort_index()
    
    # Allinea le distribuzioni per garantire lo stesso indice
    global_dist_aligned = global_distribution.reindex(global_distribution.index, fill_value=0)
    group_dist_aligned = group_distribution.reindex(global_distribution.index, fill_value=0)
    
    # Calcola la Earth Mover's Distance (EMD)
    return wasserstein_distance(global_dist_aligned, group_dist_aligned)

# Calcoliamo la t-closeness per ciascun gruppo definito dai quasi-identifier
t_closeness_values = X_test.groupby(quasi_identifiers).apply(t_closeness, global_distribution)

# Troviamo il massimo valore di t-closeness osservato (t ottimale)
t_optimal = t_closeness_values.max()

# Risultati
print(f"T-Closeness per ogni gruppo:\n{t_closeness_values}")
print(f"Valore ottimale di t (massima distanza): {t_optimal}")

T-Closeness per ogni gruppo:
age_group  gender        nationality  disability  
16-25      Female        Catalan      No              0.1884
                         English      No              0.1884
                                      Yes             0.1884
                         French       No              0.1884
                                      Not declared    0.1884
                                                       ...  
56-60      Not declared  Russian      Yes             0.1884
                         Spanish      No              0.1884
                                      Yes             0.1884
                         Swedish      No              0.1884
                                      Yes             0.1884
Length: 554, dtype: float64
Valore ottimale di t (massima distanza): 0.18839999999999998


In [79]:
results_privacy['Metric'].append("T-Closeness Max")
results_privacy['Valore'].append(t_optimal)

## Mutual Information

In [80]:
with open('Export\Variabili Modello per metriche\Variabili Modello Minimizzato\X_test_MIN.pkl', 'rb') as file:
    X_test = pickle.load(file)
    

In [81]:
X_test

,age_cv,competences_coverage_required,competences_coverage_optional,education_cv_encoded,education_job_req,education_job_encoded,gender_job_Any,gender_job_Female,gender_job_Male,gender_cv_Female,...,nationality_cv_German,nationality_cv_Irish,nationality_cv_Italian,nationality_cv_LowGerman,nationality_cv_Romanian,nationality_cv_Russian,nationality_cv_Spanish,nationality_cv_Swedish,age_min_job,age_max_job
5000,54,0.250000,0.000000,1,1,1,False,False,True,False,...,False,False,False,False,False,False,True,False,22,37
5001,26,0.500000,0.000000,2,1,1,False,False,True,True,...,False,False,False,False,False,False,True,False,22,37
5002,16,0.250000,0.000000,0,1,1,False,False,True,False,...,False,False,False,False,False,False,False,False,22,37
5003,53,1.000000,0.000000,3,1,1,False,False,True,False,...,False,False,False,False,False,False,False,False,22,37
5004,43,0.250000,0.000000,2,1,1,False,False,True,True,...,False,False,False,False,False,True,False,False,22,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,26,1.000000,0.666667,2,1,1,False,False,True,False,...,False,False,True,False,False,False,False,False,18,34
24996,32,1.000000,0.333333,2,1,1,False,False,True,False,...,False,False,False,False,False,False,True,False,18,34
24997,29,1.000000,0.333333,2,1,1,False,False,True,False,...,False,False,True,False,False,False,False,False,18,34
24998,21,0.333333,0.000000,1,1,1,False,False,True,False,...,False,False,False,False,False,False,False,False,18,34


In [82]:
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

# L'attributo sensibile o target è 'disability_cv_Yes'
y = X_test['disability_cv_Yes']

# Rimuoviamo le colonne 'disability_cv_No', 'disability_cv_Not declared' e 'disability_cv_Yes' da X_test
columns_to_exclude = ['disability_cv_No', 'disability_cv_Not declared', 'disability_cv_Yes']
X_filtered = X_test.drop(columns=columns_to_exclude, errors='ignore')  # Ignore se le colonne non esistono

# Calcolo della mutual information per le feature rimanenti rispetto a y
mutual_info = mutual_info_classif(X_filtered, y, discrete_features='auto')

# Creiamo un DataFrame per visualizzare i risultati
mutual_info_df = pd.DataFrame(mutual_info, index=X_filtered.columns, columns=['Mutual Information'])

# Calcoliamo la somma della Mutual Information
total_mi = mutual_info.sum()

# Stampa dei risultati
print(mutual_info_df)
print(f"Somma totale della Mutual Information (senza colonne escluse): {total_mi}")

                               Mutual Information
age_cv                                   0.015423
competences_coverage_required            0.016379
competences_coverage_optional            0.000000
education_cv_encoded                     0.006794
education_job_req                        0.003068
education_job_encoded                    0.000000
gender_job_Any                           0.000000
gender_job_Female                        0.008033
gender_job_Male                          0.000000
gender_cv_Female                         0.004368
gender_cv_Male                           0.015166
gender_cv_Not binary                     0.013688
gender_cv_Not declared                   0.000000
disability_job_Any                       0.005932
disability_job_No                        0.017655
PCA_1                                    0.002736
PCA_2                                    0.000000
nationality_job_Any                      0.000000
nationality_job_Catalan                  0.012768


In [83]:
results_privacy['Metric'].append("Mutual Information")
results_privacy['Valore'].append(total_mi)

In [84]:
print(results_privacy)

{'Metric': ['Avg Anonimity Set Size', 'K-Anonimity', 'L-Diversity', 'T-Closeness Max', 'Mutual Information'], 'Valore': [3.4153005464480874, 1, 1, 0.18839999999999998, 0.14116210785508043]}


In [85]:
import pandas as pd


# Trasformiamo in un DataFrame
metrics_df = pd.DataFrame(results_privacy)

# Definiamo intervalli ipotetici (min e max) per ciascuna metrica
reference_values = {
    'Avg Anonimity Set Size': {'min': 1, 'max': 100},
    'K-Anonimity': {'min': 1, 'max': 100},
    'L-Diversity': {'min': 1, 'max': 50},
    'T-Closeness Max': {'min': 0, 'max': 1},
    'Mutual Information': {'min': 0, 'max': 1},
}

# Funzione per standardizzare ciascuna metrica
def standardize_metric(row):
    metric = row['Metric']
    value = row['Valore']
    min_val = reference_values[metric]['min']
    max_val = reference_values[metric]['max']
    
    if metric in ['Avg Anonimity Set Size', 'K-Anonimity', 'L-Diversity']:
        # Inversione rispetto al massimo (valori alti migliori)
        standardized_value = (value - min_val) / (max_val - min_val)
        
    else:
        # Normalizzazione diretta (valori bassi migliori)
        standardized_value = 1 - (value - min_val) / (max_val - min_val)
    
    return standardized_value

# Applichiamo la funzione al DataFrame
metrics_df['Valore'] = metrics_df.apply(standardize_metric, axis=1)

# Riconvertiamo il DataFrame in un dizionario nella forma originale
standardized_dict = metrics_df.to_dict(orient='list')

# Stampa del dizionario finale
print(standardized_dict)

{'Metric': ['Avg Anonimity Set Size', 'K-Anonimity', 'L-Diversity', 'T-Closeness Max', 'Mutual Information'], 'Valore': [0.024396975216647347, 0.0, 0.0, 0.8116, 0.8588378921449196]}


In [86]:
with open("dicMetric_Privacy_Completed.pkl", "wb") as file:
    pickle.dump(standardized_dict, file)

In [87]:
# Converti il dizionario in un DataFrame
df = pd.DataFrame(list(standardized_dict.items()))

# Salva in un file Excel
df.to_excel("dizionario_privacy.xlsx", index=False)